![Header.jpeg](../assets/banner_linreg.png)

# Analisis dan Prediksi Konsumsi Energi pada Bangunan Menggunakan Algoritma *Ridge*, *Lasso* dan *Elastic Net Regression*

---

<h2>Daftar isi</h2>

- <a href=#bu>Business Understanding</a>
- <a href=#du>Data Understanding</a>
- <a href=#dp>Data Preparation</a>
- <a href=#ml>Modeling</a>
- <a href=#ev>Evaluation</a>
- <a href=#dy>Deployment</a>


<hr>


<h2 id='bu'>Business Understanding</h2>

<hr>

<h2 id='du'>Data Understanding</h2>

<hr>

<h2 id='du'>Data Preparation</h2>

<hr>

<h2 id='ml'>Modeling</h2>

<hr>

<h2 id='ml'>Evaluation</h2>

<hr>

<h2 id='ml'>Deployment</h2>



---


Copyright © 2025 by DGX Development Team, Gunadarma University


https://www.hpc-hub.gunadarma.ac.id/